In [30]:
import os 
import pandas as pd 
import matplotlib.pyplot as plt
from pathlib import Path
from datetime import datetime
from datetime import timedelta
import re

In [178]:
def find_latest(RESU_dir):
    date = []
    filenames = []
    dico = {}
    for filename in os.scandir(Res_dir):
        x = re.search("^[0-9]{8}-[0-9]{4}",filename.name)
        if x:
            y = re.search("^[0-9]{8}-[0-9]{4}_[0-9]",filename.name)
            if y:
                dt = datetime.strptime(filename.name[:13], "%Y%m%d-%H%M")
                dt += timedelta(0,60)
                new_name = dt.strftime("%Y%m%d-%H%M")
                os.rename(Res_dir.joinpath(filename),Res_dir.joinpath(new_name))
                fl = new_name
            else:
                dt = datetime.strptime(filename.name[:13], "%Y%m%d-%H%M")
                fl = filename.name
            filenames.append(fl)
            date.append(dt)
            dico[dt] = fl

    latest = dico[max(dico.keys())]
    last_res = RESU_dir.joinpath(latest)
    return last_res


def preprocess_listing(RESU_dir,output_name='output.txt'):
    res_dir = find_latest(RESU_dir)
    f = open(res_dir.joinpath("listing"),'r')
    f2 = open(res_dir.joinpath(output_name),'w')
    lines = f.readlines()
    start = False
    for l in lines:
        a = re.search("v  |c  |INSTANT",l)
        if not start:
            if re.search('INSTANT',l):
                start = True
        if a and start:
            f2.write(l)
    f.close()
    f2.close()
    return res_dir.joinpath(output_name)

def extract_variable(var,filepath):
    f3 = open(filepath,'r')
    lines = f3.readlines()
    ts , content = [],[]
    for l in lines:
        a = re.search("INSTANT.*NUMBER *([0-9]*)",l)
        if a:
            ts.append(int(a.group(1)))
        b = re.search(var+' *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*) *([0-9\.eE\+-]*)',l)
        if b:
            data = []
            for g in b.groups():
                if g != '':
                    data.append(float(g))
            content.append(data)
    assert(len(ts) == len(content))
    return ts,content

def extract_convergence(filepath):
    convergence = ['Rhs norm','N_iter','Norm. residual','Drift','Time residual']
    variables_c = ['Velocity','Pressure','k','epsilon']
    for i in range(len(variables_c)):
        variable = variables_c[i]
        col_names = []
        for var in convergence:
            col_names.append((variable,var))
        ts,content = extract_variable('c  '+variable,filepath)
        if i == 0:
            df = pd.DataFrame(content,index=pd.Index(ts,name="Timestep"))
            df.columns = pd.MultiIndex.from_tuples(col_names)
        else:
            df1 = pd.DataFrame(content,index=pd.Index(ts,name="Timestep"))
            df1.columns = pd.MultiIndex.from_tuples(col_names)
            df = df.join(df1)
    df = df.swaplevel(axis=1)
    df.sort_index(axis=1)
    return df

def extract_values(filepath):
    fields = ['minimum','maximum','set_mean','spatial_mean']
    variables_v = ['ǁVelocityǁ','Pressure','k','epsilon','CourantNb']
    for i in range(len(variables_v)):
        variable = variables_v[i]
        col_names = []
        for var in fields:
            col_names.append((variable,var))
        ts,content = extract_variable('v  '+variable,filepath)
        if i == 0:
            df = pd.DataFrame(content,index=pd.Index(ts,name="Timestep"))
            df.columns = pd.MultiIndex.from_tuples(col_names)
        else:
            df1 = pd.DataFrame(content,index=pd.Index(ts,name="Timestep"))
            df1.columns = pd.MultiIndex.from_tuples(col_names)
            df = df.join(df1)
    df = df.swaplevel(axis=1)
    df = df.sort_index(axis=1)
    return df

def process_listing(res_directory):
    temporary_file = 'output.txt'
    resu_path = preprocess_listing(res_directory,temporary_file)
    df_c = extract_convergence(resu_path)
    df_v = extract_values(resu_path)
    return df_c,df_v

In [179]:
def process_residuals(last_res):
    df_res = pd.read_csv(last_res.joinpath("residuals.csv"))
    fig,[[ax1,ax2],[ax3,ax4]] = plt.subplots(nrows = 2,ncols = 2)
    df_res.plot(ax=ax1,x="t",y=" velocity")
    df_res.plot(ax=ax2,x="t",y=" pressure")
    df_res.plot(ax=ax3,x="t",y=" k")
    df_res.plot(ax=ax4,x="t",y=" epsilon")

In [180]:
Doc_dir = Path("/home/juleslecoustre/Documents")
Study_name = "STUDY"
Case_Name = "FLOW"

Case_dir = Doc_dir.joinpath(Study_name).joinpath(Case_Name)
Res_dir = Case_dir.joinpath("RESU")

In [182]:
df1,df2 = process_listing(Res_dir)
df1

,Rhs norm,N_iter,Norm. residual,Drift,Time residual,Rhs norm,N_iter,Norm. residual,Drift,Time residual,Rhs norm,N_iter,Norm. residual,Drift,Time residual,Rhs norm,N_iter,Norm. residual,Drift,Time residual
,Velocity,Velocity,Velocity,Velocity,Velocity,Pressure,Pressure,Pressure,Pressure,Pressure,k,k,k,k,k,epsilon,epsilon,epsilon,epsilon,epsilon
Timestep,,,,,,,,,,,,,,,,,,,,
1,180.26,30.0,0.007496,0.045334,0.67187,1.3523,34.0,0.000176,0.010912,10.0000,1.550500e+00,35.0,1.254400e-03,7.030500e-06,4.7446,3.707500e-01,34.0,2.419300e-04,4.199200e-07,9.9800
2,203.09,46.0,0.051908,0.052510,0.72035,408.8500,39.0,0.003252,0.976960,9.0759,2.408100e+00,42.0,1.391600e-03,4.185800e-06,2.9238,7.976400e+04,1.0,2.491500e-07,1.832700e-07,4.1559
3,5164.20,35.0,0.056081,92.746000,9.40260,10082.0000,38.0,0.001571,0.992670,9.8372,3.197400e+00,31.0,4.689600e-04,4.027600e-06,2.3761,1.100300e+10,26.0,4.623500e-03,3.036500e+14,10.0000
4,5341.10,35.0,0.538060,39.018000,9.68970,5193.0000,42.0,0.003719,1.164900,16.4370,6.492700e+07,1.0,4.259800e-09,6.579700e-05,31.7210,8.537400e+24,1.0,8.411500e-11,3.190200e+27,10.0000
5,2670.60,32.0,0.301000,7.279200,5.22310,2802.0000,36.0,0.002831,0.913390,7.2606,1.160400e+22,1.0,5.473800e-24,5.166500e-06,20.0330,1.662300e+37,1.0,4.538700e-13,3.194700e+27,262.5100
6,2429.60,30.0,0.381760,3.968100,4.26980,1762.5000,38.0,0.002442,0.713600,10.5600,8.423300e+27,1.0,1.962300e-29,1.428100e-06,6.3728,7.422300e+41,1.0,1.476000e-16,4.636100e+24,7698.5000
7,2330.30,29.0,0.703560,2.992000,4.17760,1224.7000,40.0,0.001998,0.635000,11.1640,2.478900e+23,1.0,5.131900e-25,4.658100e-06,10.5210,1.050000e+34,1.0,1.449400e-14,7.839800e+18,212.3300
8,3762.80,27.0,0.786190,4.067600,4.79330,1214.3000,37.0,0.001928,0.703860,6.6630,1.084300e+22,1.0,2.944400e-23,2.890500e-06,6.7446,2.948700e+31,1.0,1.574300e-16,2.339000e+19,9.9997
